# Guía generación BSC

Para generar un **informe de Balanced Scorecard en Power BI** a partir de la base de datos que has compartido, necesitamos:

1. Extraer los datos relevantes.
2. Modelar las tablas y crear métricas clave (KPIs).
3. Diseñar visualizaciones por cada perspectiva del BSC:
   - Financiera
   - Clientes
   - Procesos internos
   - Aprendizaje y crecimiento

Como no podemos generar directamente un archivo `.pbix` desde este entorno, te voy a guiar paso a paso para que puedas construirlo tú mismo en Power BI Desktop.



## 1. Cargar los datos en Power BI

### Opción 1: Conectar a PostgreSQL
- Abre **Power BI Desktop**.
- Haz clic en **Obtener datos > Más... > PostgreSQL**.
- Ingresa los datos de conexión a tu base de datos.
- Selecciona las tablas: `customers`, `employees`, `payments`, `products`, `sales`.
- Haz clic en **Cargar**.

### Opción 2: Usar archivos CSV (si prefieres trabajar offline)
Puedes exportar tus tablas como CSV y cargarlas en Power BI.



## 2. Modelo de datos

Una vez cargadas las tablas, crea relaciones:

| Relación | Campo izquierdo | Campo derecho |
|----------|------------------|----------------|
| `sales → customers` | `sales.customerNumber` | `customers.customerNumber` |
| `sales → employees` | `sales.employeeNumber` | `employees.employeeNumber` |
| `sales → products` | `sales.productCode` | `products.productCode` |
| `payments → customers` | `payments.customerNumber` | `customers.customerNumber` |

> En **Modelado → Relaciones**, define estas conexiones.



## 3. Crear medidas (KPIs)

Ve a la pestaña **Modelado → Nueva medida** y crea las siguientes:

### Ventas totales

```dax
Total Sales = SUMX(sales, sales.quantityOrdered * sales.priceEach)
```

### Ingresos totales (ventas + pagos)

```dax
Total Revenue = 
IF(
    ISINSCOPE(sales),
    [Total Sales],
    SUM(payments.amount)
)
```

### Margen de beneficio (si tienes costos; si no, ignora o usa placeholder)

```dax
Profit Margin = ([Total Sales] - [Total Cost]) / [Total Sales]
```

### Satisfacción del cliente (ejemplo basado en comentarios positivos o promedio de rating si lo tuvieras)

```dax
Customer Satisfaction = AVERAGE(customers.rating) // Requiere campo rating
```

### Productividad del empleado (ventas por empleado)

```dax
Sales per Employee = DIVIDE([Total Sales], DISTINCTCOUNT(employees.employeeNumber))
```



## 4. Diseño del Balanced Scorecard

### Perspectiva: **Financiera**

- Visualización: Tarjetas o gráfico de líneas
  - Total de ventas
  - Ingresos totales
  - Margen de beneficio

### Perspectiva: **Clientes**

- Visualizaciones:
  - Número de clientes únicos: `DISTINCTCOUNT(customers.customerNumber)`
  - Ventas por cliente (gráfico de barras)
  - Promedio de ventas por cliente: `[Total Sales] / DISTINCTCOUNT(customers.customerNumber)`

### Perspectiva: **Procesos Internos**

- Ejemplos:
  - Tiempo promedio entre pedido y envío: `AVERAGE(DATEDIFF(sales.orderDate, sales.shippedDate, DAY))`
  - Tasa de cumplimiento de fechas de entrega: `% de órdenes entregadas antes de requiredDate`

### Perspectiva: **Aprendizaje y Crecimiento**

- Ejemplos:
  - Número de empleados nuevos por mes
  - Ventas por empleado (medida creada antes)
  - Jerarquía de reportes (`reportsTo`) para mostrar estructura organizacional



## 5. Diseño final

Diseña una página con título “Balanced Scorecard” y divide en secciones:

| Sección | Contenido |
|--------|-----------|
| **Encabezado** | Filtros globales (por región, año, empleado, cliente) |
| **Columna Izquierda** | Perspectivas financieras |
| **Columna Centro-Izquierda** | Perspectiva de clientes |
| **Columna Centro-Derecha** | Perspectiva de procesos internos |
| **Columna Derecha** | Perspectiva de aprendizaje y crecimiento |

Usa tarjetas, gráficos de barras horizontales, KPIs y mapas visuales.



## 6. Exportar el informe

Una vez terminado:

- Haz clic en **Archivo > Guardar como**.
- Guarda como `Balanced_Scorecard.pbix`.

También puedes publicarlo en **Power BI Service** para compartirlo con otros usuarios.
